# SiliconCrowds Pipeline Tutorial

This notebook demonstrates how to use the SiliconCrowds platform to run multimodal AI predictions on Norwegian crowd prediction tasks.

**What you'll learn:**
- How to load and configure vision language models from Fireworks.ai
- How to access question contexts (transcripts + images) from the database
- How to send multimodal prompts to the model and interpret responses

**Prerequisites:**
- `.env` file configured with `FIREWORKS_API_KEY`, `SUPABASE_URL`, and `SUPABASE_KEY`
- Dependencies installed via `uv sync`

## Load Libraries

Import the core components:
- `Model`: Wrapper for Fireworks.ai vision language models
- `Contextual`: Aggregates questions from the database with their corresponding images
- `Instruction`, `Prompt`: Manages prompt templates and builds message lists
- `Message`: Pydantic model representing a single chat message

In [1]:
from rich import print as rich_print
from siliconcrowds.model import Model, Config
from siliconcrowds.context import Contextual, Context
from siliconcrowds.prompt import Instruction, Prompt
from siliconcrowds.model import Message
from siliconcrowds.schema import NumericSchema, TimeSchema

## Load Model

Select a vision language model from the [Fireworks.ai Model Library](https://app.fireworks.ai/models). 

For multimodal tasks (text + image), use models with vision capabilities such as:
- `qwen3-vl-30b-a3b-thinking` (used here)
- `llama4-maverick-instruct-basic`
- `phi-4-multimodal-instruct`

### Configuration

The `Config` class controls model behavior:
- **temperature**: Controls randomness in responses (0.0 = deterministic, 1.0 = creative). Default is 0.1 for consistent predictions.

**Note:** Configuration parameters must be supported by the specific model you are using. Check the [Fireworks.ai documentation](https://docs.fireworks.ai/) for model-specific parameter support.

In [2]:
config = Config(temperature=0.1)
model = Model(model="accounts/fireworks/models/qwen3-vl-30b-a3b-thinking", config=config)

## Load Prompt

The `Instruction` class loads prompt templates from the database, organized by category:
- `baseline`: Standard prompts without persona simulation
- `generic_persona`: Prompts for generic demographic role-play
- `specific_persona`: Prompts for specific demographic combinations

Each `Prompt` contains a `system_prompt` (model behavior) and `user_prompt` (template with `{transcript}` and `{image}` placeholders).

In [3]:
instruction = Instruction()
print("================ Baseline Prompt: ==================")
prompt: Prompt = instruction.get_baseline_prompt("baseline_instructional_1")
rich_print(prompt)

================ Baseline Prompt: ==================


Prompt(
    id=37,
    category='baseline',
    system_prompt="You are predicting the outcome of a TV game-show experiment.\n\n\nIn the show, playful 
real-world experiments are presented, and a numeric prediction about the outcome must be made.\n\nYou will receive 
a written transcript of the experiment (in Norwegian) and an image from the experiment. Use both to produce the 
numeric guess of the experiment's outcome.\n\nRespond with exactly one numeric value (integer, decimal, or 
formatted as mm:ss). Nothing else.",
    user_prompt='###TRANSCRIPT###\n{transcript}\n\n###IMAGE###\n{image}\n\n\nYour answer:',
    template_name='baseline_instructional_1',
    description=None
)

## Load Context

The `Contextual` class loads all questions from the Supabase database and matches them with signed image URLs from storage.

Each `Context` contains:
- **prompt.transcript**: The Norwegian TV show transcript describing the task
- **prompt.image_url**: A signed URL to the associated image (valid for 5 minutes)
- **answer.norways_answer**: What Norway collectively predicted
- **answer.actual_outcome**: The real outcome
- **answer.answer_type**: Type of answer expected (e.g., "numeric")

In [4]:
contextual = Contextual()
ids: list[str] = contextual.get_ids()
context: Context = contextual[ids[0]]

print("================ Context: ==================")
rich_print(context)

Storage endpoint URL should have a trailing slash.
================ Context: ==================


Context(
    id='5',
    question_id='S5E7_Q3',
    prompt=ContextPrompt(
        transcript='Også i kveld skal vi da til bobbanen på Lillehammer, for å gi nok en idrettsutøver en 
skikkelig, skikkelig utfordring.\nVi er tilbake\ni ett av Norges råeste idrettsanlegg,\nnemlig bob-banen\ni 
Olympiaparken på Lillehammer.\nDenne gangen skal vi gjøre noe jeg er\nganske sikker på at ingen har gjort før.\nVi 
har kjørt ned her i hjulbob og sendt\nutfor folk på både longboard og sykkel.\nMen i dag skal vi gjøre noe litt 
annet.\nDette er Torstein Aanekre.\nHan har vært norgesmester i rullestolbasket,\nog er på landslaget i 
kjelkehockey.\nTorstein er godt vant til å bruke armene\nfor å komme seg raskt fremover.\nOg det kommer han til å 
få ordentlig god bruk for i dag.\nHalla.\nHalla.\nHva har du sagt ja til nå?\nNei, det er jeg litt usikker på 
selv.\nNå tenker dere kanskje\nat nå skal vi jaggu meg sende Torstein ned her.\nDet hadde jo blitt altfor 
enkelt.\nMen vi skal gjøre jeg syntes er enda voldsommere.\nVi skal sende deg opp.\nTorstein skal nemlig 
starte\nved det som til vanlig er målstreken,\nog kjempe seg opp de 1283 meterne,\ntil det som vanligvis er 
startstreken.\nOpp dit er det 114 høydemeter.\nTil sammenligning er dette 3 meter kortere\nenn helt opp til toppen 
av Oslo Plaza.\nDa Victor og jeg kjørte ned i hjulbob,\nble tiden 1 min og 6 sek.\nMen det var riktignok nedover 
banen.\nEr det å jobbe og trene i oppoverbakke\nen del av hvordan du trener til vanlig?\nDet er en del i 
motbakker,\nmen da holder vi på i kanskje 30 sekunder med intervaller i motbakker.\nDette her blir jo 
seigpining.\nDet verste er at det er brattest på slutten.\nHva tenker du om løypa, da?\nMed alle svingene og 
sånn?\nJeg var litt nervøs for det, faktisk.\nMen jeg har vært og testa det,\nog det er flatt og fint, selv i 
svingene.\nSå det var ikke noe problem.\nRullestolen veier 5 kg.\nHjulene er på 24 tommer, altså 61 cm i 
diameter.\nTorstein får lov til\nå stoppe og hvile om han trenger det, det er veldig forståelig\nmen klokken 
stopper ikke før han krysser\nmålstreken - eller startstreken, alt ettersom.\nJeg slutter aldri å bli 
imponert\nover hva folk utsetter seg for\ni lørdagsunderholdningens\nog vitenskapens navn.\nHer er 
spørsmålet:\nHvor lang tid bruker Torstein\npå å komme seg opp bob- banen?\nSvaret ligger et sted mellom 0 og 60 
minutter.',
        image_url='https://mlttcybnjagqybcwespv.supabase.co/storage/v1/object/sign/pilot_images/pilot_images/S5E7_Q
3.png?token=eyJraWQiOiJzdG9yYWdlLXVybC1zaWduaW5nLWtleV8xMTEzMGYzMC0yNDgyLTRkNDktYWY1OS0yMTYyMjMxZWVhYjEiLCJhbGciOiJ
IUzI1NiJ9.eyJ1cmwiOiJwaWxvdF9pbWFnZXMvcGlsb3RfaW1hZ2VzL1M1RTdfUTMucG5nIiwiaWF0IjoxNzcwMDY4ODA5LCJleHAiOjE3NzAwNzA2M
Dl9.BSE42va0RboWSYnRZTdPahB-TFsNTx-WBQg8q4QI6EI'
    ),
    answer=Answer(norways_answer='29:57', actual_outcome='23:51', answer_type='time_mm:ss')
)

## Build Message List

The `Instruction.build_message()` method transforms the prompt template and context into a list of messages that the LLM can process.

**What it does:**
1. Creates a system message from `system_prompt`
2. Formats the user prompt by inserting the transcript
3. Removes the `###IMAGE###` placeholder from the text
4. Adds the image as a separate message (if provided)

**Why move the image?** The database stores prompts as text templates with `###IMAGE###` placeholders, but vision-language models require images as dedicated content blocks with `type: image_url`. This function bridges that gap.

**No images?** Just omit the `image_url` parameter and the image message will not be added to the list.

In [5]:
messages: list[Message] = Instruction.build_message(prompt=prompt, transcript=context.prompt.transcript, image_url=context.prompt.image_url)

print("================ Message: ==================")
rich_print(messages)

================ Message: ==================


[
    Message(
        role=<MessageRole.SYSTEM: 'system'>,
        content=[
            {
                'type': 'text',
                'text': "You are predicting the outcome of a TV game-show experiment.\n\n\nIn the show, playful 
real-world experiments are presented, and a numeric prediction about the outcome must be made.\n\nYou will receive 
a written transcript of the experiment (in Norwegian) and an image from the experiment. Use both to produce the 
numeric guess of the experiment's outcome.\n\nRespond with exactly one numeric value (integer, decimal, or 
formatted as mm:ss). Nothing else."
            }
        ]
    ),
    Message(
        role=<MessageRole.USER: 'user'>,
        content=[
            {
                'type': 'text',
                'text': '###TRANSCRIPT###\nOgså i kveld skal vi da til bobbanen på Lillehammer, for å gi nok en 
idrettsutøver en skikkelig, skikkelig utfordring.\nVi er tilbake\ni ett av Norges råeste idrettsanlegg,\nnemlig 
bob-banen\ni Olympiaparken på Lillehammer.\nDenne gangen skal vi gjøre noe jeg er\nganske sikker på at ingen har 
gjort før.\nVi har kjørt ned her i hjulbob og sendt\nutfor folk på både longboard og sykkel.\nMen i dag skal vi 
gjøre noe litt annet.\nDette er Torstein Aanekre.\nHan har vært norgesmester i rullestolbasket,\nog er på 
landslaget i kjelkehockey.\nTorstein er godt vant til å bruke armene\nfor å komme seg raskt fremover.\nOg det 
kommer han til å få ordentlig god bruk for i dag.\nHalla.\nHalla.\nHva har du sagt ja til nå?\nNei, det er jeg litt
usikker på selv.\nNå tenker dere kanskje\nat nå skal vi jaggu meg sende Torstein ned her.\nDet hadde jo blitt 
altfor enkelt.\nMen vi skal gjøre jeg syntes er enda voldsommere.\nVi skal sende deg opp.\nTorstein skal nemlig 
starte\nved det som til vanlig er målstreken,\nog kjempe seg opp de 1283 meterne,\ntil det som vanligvis er 
startstreken.\nOpp dit er det 114 høydemeter.\nTil sammenligning er dette 3 meter kortere\nenn helt opp til toppen 
av Oslo Plaza.\nDa Victor og jeg kjørte ned i hjulbob,\nble tiden 1 min og 6 sek.\nMen det var riktignok nedover 
banen.\nEr det å jobbe og trene i oppoverbakke\nen del av hvordan du trener til vanlig?\nDet er en del i 
motbakker,\nmen da holder vi på i kanskje 30 sekunder med intervaller i motbakker.\nDette her blir jo 
seigpining.\nDet verste er at det er brattest på slutten.\nHva tenker du om løypa, da?\nMed alle svingene og 
sånn?\nJeg var litt nervøs for det, faktisk.\nMen jeg har vært og testa det,\nog det er flatt og fint, selv i 
svingene.\nSå det var ikke noe problem.\nRullestolen veier 5 kg.\nHjulene er på 24 tommer, altså 61 cm i 
diameter.\nTorstein får lov til\nå stoppe og hvile om han trenger det, det er veldig forståelig\nmen klokken 
stopper ikke før han krysser\nmålstreken - eller startstreken, alt ettersom.\nJeg slutter aldri å bli 
imponert\nover hva folk utsetter seg for\ni lørdagsunderholdningens\nog vitenskapens navn.\nHer er 
spørsmålet:\nHvor lang tid bruker Torstein\npå å komme seg opp bob- banen?\nSvaret ligger et sted mellom 0 og 60 
minutter.Your answer:'
            }
        ]
    ),
    Message(
        role=<MessageRole.USER: 'user'>,
        content=[
            {
                'type': 'image_url',
                'image_url': {
                    'url': 
'https://mlttcybnjagqybcwespv.supabase.co/storage/v1/object/sign/pilot_images/pilot_images/S5E7_Q3.png?token=eyJraW
QiOiJzdG9yYWdlLXVybC1zaWduaW5nLWtleV8xMTEzMGYzMC0yNDgyLTRkNDktYWY1OS0yMTYyMjMxZWVhYjEiLCJhbGciOiJIUzI1NiJ9.eyJ1cmwi
OiJwaWxvdF9pbWFnZXMvcGlsb3RfaW1hZ2VzL1M1RTdfUTMucG5nIiwiaWF0IjoxNzcwMDY4ODA5LCJleHAiOjE3NzAwNzA2MDl9.BSE42va0RboWSY
nRZTdPahB-TFsNTx-WBQg8q4QI6EI'
                }
            }
        ]
    )
]

## Structured Output

Structured output ensures the model returns responses in a predefined format that can be reliably parsed and validated. This is essential for the pipeline because:

1. **Type Safety**: Responses are automatically validated against a Pydantic schema, catching format errors immediately
2. **Consistent Parsing**: No need for regex or string parsing to extract answers from free-form text
3. **Schema Enforcement**: The model is constrained to output valid JSON matching the schema

### Available Schemas

The pipeline supports two answer types matching the question formats:

| Schema | Format | Example | Use Case |
|--------|--------|---------|----------|
| `NumericSchema` | Integer | `{"answer": 42}` | Count-based questions (e.g., "How many people...") |
| `TimeSchema` | mm:ss string | `{"answer": "29:57"}` | Duration questions (e.g., "How long did it take...") |

### How It Works

When you pass a schema to `model.invoke()`:
1. The API adds `response_format: {"type": "json_object", "schema": ...}` to the request
2. The model generates JSON conforming to the schema
3. The response is automatically parsed into a Pydantic object accessible via `response.structured_output`

### Retry Logic

Even with schema enforcement, models occasionally produce responses that fail validation (e.g., returning `"25.5"` instead of `"25:30"` for a time format). The `retries` parameter handles this automatically:

1. If validation fails, the model's invalid response is appended to the conversation
2. A correction message explaining the validation error is sent back to the model
3. The model attempts to generate a valid response

Set `retries` at initialization (`Model(..., retries=2)`) for a default, or per-call (`model.invoke(..., retries=3)`) to override. The default is 2 attempts (1 initial + 1 retry).

### Schema Selection

The schema is selected based on `context.answer.answer_type`:
- `"numeric"` → `NumericSchema` (integer answer)
- `"time_mm:ss"` → `TimeSchema` (time string in mm:ss format)

In [6]:
answer_type: str | None = context.answer.answer_type

if answer_type == "numeric":
    StructuredOutputSchema: type[NumericSchema] = NumericSchema
elif answer_type == "time_mm:ss":
    StructuredOutputSchema: type[TimeSchema] = TimeSchema
else:
    raise ValueError(f"Unknown answer_type: {answer_type}. Expected 'numeric' or 'time_mm:ss'.")

print("================ Structured Output Schema: ==================")
rich_print(StructuredOutputSchema.model_json_schema())

================ Structured Output Schema: ==================


{
    'properties': {
        'answer': {
            'description': "Time duration in mm:ss format (e.g., '00:48', '29:57'). mm:ss corresponds to 
minutes:seconds. Only use for questions asking how long something takes or how much time passes. Examples of time 
questions: 'How long will it take to accomplish the task?', 'How much time does it take to go from A to B?'",
            'pattern': '^\\d{1,2}:[0-5]\\d$',
            'title': 'Answer',
            'type': 'string'
        }
    },
    'required': ['answer'],
    'title': 'TimeSchema',
    'type': 'object'
}

## Run Pipeline

Send the message list to the model with the selected schema. The vision-language model processes both the transcript and image to generate a prediction.

**Parameters:**
- `structured_output`: The Pydantic schema that enforces the response format
- `retries`: Number of attempts if validation fails (default: 2)

**Response fields:**
- `message.content`: The model's raw JSON response
- `structured_output`: The parsed Pydantic object with the answer
- `reasoning_content`: The model's reasoning process (for thinking models)
- `usage`: Token counts for cost tracking

In [7]:
response = model.invoke(messages, structured_output=StructuredOutputSchema, retries=2)

print("================ Response: ==================")
rich_print(response)

================ Response: ==================


Response(
    id='501d3378-606b-4644-bf44-3704f343d9d6',
    message=Message(
        role=<MessageRole.ASSISTANT: 'assistant'>,
        content=[{'type': 'text', 'text': '{"answer": "25:30"}\n'}]
    ),
    reasoning_content=None,
    model='accounts/fireworks/models/qwen3-vl-30b-a3b-thinking',
    usage=Usage(prompt_tokens=2293, completion_tokens=11, total_tokens=2304),
    structured_output=TimeSchema(answer='25:30')
)

## Interpret Results

Compare the model's prediction against:
- **Norway's answer**: What the Norwegian population collectively predicted
- **Actual outcome**: What actually happened

This helps evaluate how well the model simulates crowd behavior.

In [8]:
model_prediction = response.structured_output
norways_answer = context.answer.norways_answer
actual_outcome = context.answer.actual_outcome

print("================ Comparison: ==================")
print(f"Model's prediction:  {model_prediction.answer}")
print(f"Norway's answer:     {norways_answer}")
print(f"Actual outcome:      {actual_outcome}")

================ Comparison: ==================
Model's prediction:  25:30
Norway's answer:     29:57
Actual outcome:      23:51
